In [33]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error

In [38]:
data = pd.read_csv('/home/ganesh/Documents/Datasets/allYearHarangi.csv')
data=data.drop(columns=['Dates','Present Storage(TMC)','Reservoir Level(TMC)','Outflow'])
data

,Inflow,MADIKERI,SOMWARPET,VIRAJPET
0,142.0,0.0,0.0,0.0
1,239.0,0.0,0.0,0.0
2,166.0,0.0,0.0,0.0
3,118.0,0.0,0.0,0.0
4,130.0,0.0,0.0,0.0
...,...,...,...,...
2917,103.0,0.0,0.0,0.0
2918,98.0,0.0,0.0,0.0
2919,102.0,0.0,0.0,0.0
2920,99.0,0.0,0.0,0.0


In [138]:
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
	n_vars = 1 if type(data) is list else data.shape[1]
	df = pd.DataFrame(data)
	cols, names = list(), list()
	# input sequence (t-n, ... t-1)
	for i in range(n_in, 0, -1):
		cols.append(df.shift(i))
		names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
	# forecast sequence (t, t+1, ... t+n)
	for i in range(0, n_out):
		cols.append(df.shift(-i))
		if i == 0:
			names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
		else:
			names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
	# put it all together
	agg = pd.concat(cols, axis=1)
	agg.columns = names
	# drop rows with NaN values
	if dropnan:
		agg.dropna(inplace=True)
	return agg
 
# load dataset
# dataset = pd.read_csv('pollution.csv', header=0, index_col=0)
values = data.values
# integer encode direction
# encoder = LabelEncoder()
# values[:,4] = encoder.fit_transform(values[:,4])
# ensure all data is float
values = values.astype('float32')
# normalize features
scaler = MinMaxScaler(feature_range=(0, 1))
scaled = scaler.fit_transform(values)
print(scaled)
# frame as supervised learning
reframed = series_to_supervised(scaled, 3, 1)
# drop columns we don't want to predict
#reframed.drop(reframed.colum, axis=1, inplace=True)
reframed.head()

[[0.00182285 0.         0.         0.        ]
 [0.00306804 0.         0.         0.        ]
 [0.00213094 0.         0.         0.        ]
 ...
 [0.00130937 0.         0.         0.        ]
 [0.00127086 0.         0.         0.        ]
 [0.00127086 0.         0.         0.        ]]


,var1(t-3),var2(t-3),var3(t-3),var4(t-3),var1(t-2),var2(t-2),var3(t-2),var4(t-2),var1(t-1),var2(t-1),var3(t-1),var4(t-1),var1(t),var2(t),var3(t),var4(t)
3,0.001823,0.0,0.0,0.0,0.003068,0.0,0.0,0.0,0.002131,0.0,0.0,0.0,0.001515,0.0,0.0,0.0
4,0.003068,0.0,0.0,0.0,0.002131,0.0,0.0,0.0,0.001515,0.0,0.0,0.0,0.001669,0.0,0.0,0.0
5,0.002131,0.0,0.0,0.0,0.001515,0.0,0.0,0.0,0.001669,0.0,0.0,0.0,0.002144,0.0,0.0,0.0
6,0.001515,0.0,0.0,0.0,0.001669,0.0,0.0,0.0,0.002144,0.0,0.0,0.0,0.001913,0.0,0.0,0.0
7,0.001669,0.0,0.0,0.0,0.002144,0.0,0.0,0.0,0.001913,0.0,0.0,0.0,0.001682,0.0,0.0,0.0


In [120]:
reframed

,var1(t-3),var2(t-3),var3(t-3),var4(t-3),var1(t-2),var2(t-2),var3(t-2),var4(t-2),var1(t-1),var2(t-1),var3(t-1),var4(t-1),var1(t),var2(t),var3(t),var4(t)
3,0.001823,0.0,0.0,0.0,0.003068,0.0,0.0,0.0,0.002131,0.0,0.0,0.0,0.001515,0.0,0.0,0.0
4,0.003068,0.0,0.0,0.0,0.002131,0.0,0.0,0.0,0.001515,0.0,0.0,0.0,0.001669,0.0,0.0,0.0
5,0.002131,0.0,0.0,0.0,0.001515,0.0,0.0,0.0,0.001669,0.0,0.0,0.0,0.002144,0.0,0.0,0.0
6,0.001515,0.0,0.0,0.0,0.001669,0.0,0.0,0.0,0.002144,0.0,0.0,0.0,0.001913,0.0,0.0,0.0
7,0.001669,0.0,0.0,0.0,0.002144,0.0,0.0,0.0,0.001913,0.0,0.0,0.0,0.001682,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2917,0.001258,0.0,0.0,0.0,0.001438,0.0,0.0,0.0,0.001258,0.0,0.0,0.0,0.001322,0.0,0.0,0.0
2918,0.001438,0.0,0.0,0.0,0.001258,0.0,0.0,0.0,0.001322,0.0,0.0,0.0,0.001258,0.0,0.0,0.0
2919,0.001258,0.0,0.0,0.0,0.001322,0.0,0.0,0.0,0.001258,0.0,0.0,0.0,0.001309,0.0,0.0,0.0
2920,0.001322,0.0,0.0,0.0,0.001258,0.0,0.0,0.0,0.001309,0.0,0.0,0.0,0.001271,0.0,0.0,0.0


In [170]:
train_y = data_values[:,3]
train_y.shape

(2919, 4)

In [44]:
def prediction(x,params):
    predict = 0
    for weight, features in zip(params["w"],x):
        predict += weight*features
    predict += params['b']
    return predict

In [199]:
data_values = np.array(reframed)
data_values = data_values.reshape(2919,4,4)
train_x = data_values[:,:3]
train_x = train_x.reshape(2919,3,4,1)
# train_x = train_x.transpose(0,2,1)
# train_x.shape
train_x.shape

(2919, 3, 4, 1)

In [159]:
w=[]
w.append(np.random.uniform(size=(4,4)))
w.append(np.random.uniform(size=(4,4)))
w.append(np.random.uniform(size=(4,4)))
bias= np.random.uniform(size=(4,1))

In [201]:
w[0]
out.shape

(4, 1)

In [204]:
for epoch in range(10):
    #out = bias
    for i in range(3):
        out += w[i].dot(train_x[:,i].T)
        print(out)
        
#     for j in range(3):
#         descent(out,train_x[j],train_y,w[j])
        
    #out = (bias[i] +  (w[i] * var[i])) #This is the basic structure 

ValueError: non-broadcastable output operand with shape (4,1) doesn't match the broadcast shape (4,4,2919)

In [166]:
w[0].dot(train_x[0].T)

array([[5.48084815e-05, 9.22480814e-05, 6.40718883e-05],
       [1.60806775e-03, 2.70653666e-03, 1.87985389e-03],
       [6.20232685e-04, 1.04391280e-03, 7.25060761e-04],
       [1.61355266e-03, 2.71576831e-03, 1.88626583e-03]])

In [118]:
def input_values(data_values,batch_number):
    
i =-1
var = []
for j in range(b):
    x = j%4
    if x == 0:
        var.append(list())
        i = i+1
    var[i].append(data_values[0][j])    
var

IndentationError: expected an indented block (<ipython-input-118-aaef00850dbd>, line 2)

[[0.0018228497, 0.0, 0.0, 0.0],
 [0.003068036, 0.0, 0.0, 0.0],
 [0.002130937, 0.0, 0.0, 0.0],
 [0.0015147625, 0.0, 0.0, 0.0]]

In [24]:
data.iloc[18+4,3]

0.0

In [25]:
xx=np.array(xx)
y=np.array(y)
y.shape
xx.shape

(2900, 3, 2)

In [38]:

xx[176]

array([[  41.16666667, 5117.        ],
       [  24.25      , 4004.        ],
       [  24.        , 6761.        ]])

In [103]:
out = bias+np.dot(w[0],xx[:,0].T)+np.dot(w[1],xx[:,1].T)+np.dot(w[2],xx[:,2].T)

In [60]:
(bias.T+xx[:,0]).shape

(2400, 2)

In [65]:
np.dot(w[0],xx[:,0].T).shape

(2, 2400)

In [50]:
from sklearn import metrics

In [70]:
xx[967,0].T

array([   5.825, 4942.   ])

In [75]:
out[0]

array([ 353.47083255,  332.62034435,  272.40974041, ..., 6306.40610643,
       7462.13850505, 7388.89777237])

In [211]:
xx[:,0].T.shape

(2, 2900)

In [174]:
import math

In [245]:
math.sqrt(metrics.mean_squared_error(y.T,out))

3.40713756343876e+19

In [81]:
# def sigmoid (x):
#     return (1/(1 + np.exp(-x)))
# def der(x):
#     return (x*(1-x))

In [242]:
#out=np.zeros((2,2400),dtype=np.float64)

In [27]:
def prediction(x,params):
    predict = 0
    for weight, features in zip(params["w"],x):
        predict += weight*features
    predict += params['b']
    return predict

In [ ]:
def train(X,y,noob_val,learning_rate = 0.0005,iterations = 20000):
    
    print(mae([prediction(X,noob_val) for x in X],y))
    print(print(noob_val))
    print('prior gradient descent')
    for i in range(iterations):
        acc_grad_w = 0
        acc_grad_b = 0
        for x,y_target in zip(X,y):
            acc_grad_w += (prediction(x,noob_val) - y_target)*x[0]
            acc_grad_b += (prediction(x,noob_val) - y_target)
        grad_mean_w = (1/len(X)) * acc_grad_w
        grad_mean_b = (1/len(X)) * acc_grad_b

        noob_val['w'][0] = noob_val['w'][0] - learning_rate * grad_mean_w 
        noob_val['b'] = noob_val['b'] - learning_rate * grad_mean_b
        
        if i%4000 == 0:
            print("While Gradient descent")
            print(i)
            print(mae([prediction(X,noob_val) for x in X],y))
            print(print(noob_val))
            
    print("After Gradient Descent")       
    print(mae([prediction(X,noob_val) for x in X],y))
    print(print(noob_val))

In [28]:
def mae(prediction,actual):
    num_samples = len(prediction)
    sums=0.0
    
    for predict,actual in zip(prediction,actual):
        sums += (predict - actual)**2 
    
    final_res = (1.0/num_samples)*sums
    return final_res

In [32]:
w=[]
w = np.random.uniform(size=(2,2))
bias= np.random.uniform(size=(2,1))
w[0]

array([0.83321109, 0.77039904])

In [247]:
x=0
for i in range(5):
    

In [214]:
err.shape

(2, 2900)

In [206]:
 xx[:,0].shape

(2900, 2)

In [140]:
    out = bias+np.dot(w[0],xx[:,0].T)+np.dot(w[1],xx[:,1].T)+np.dot(w[2],xx[:,2].T)
    x=(y.T-out)
    err = der(out)*x
    for i in range(len(w)):
        w[i]-=xx[:,i].T.dot(err.T)*0.2

/home/kishora/PycharmProjects/abc/venv/lib/python3.6/site-packages/ipykernel_launcher.py:4: RuntimeWarning: overflow encountered in multiply
  after removing the cwd from sys.path.


In [138]:
pd.DataFrame(out).isna()

,0,1,2,3,4,5,6,7,8,9,...,2390,2391,2392,2393,2394,2395,2396,2397,2398,2399
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [122]:
out.shape

(2, 2400)

In [123]:
out

array([[nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan]])

In [147]:
inp=[]
y=[]
for i in range(2400,2900):
    z=[]
    X=[]
    for j in range(i,i+3):
        a=[]
        s=[]
        s.append(data.iloc[j,-1])
        s.append(data.iloc[j,3])
        a.append(s)
        
        X.extend(a)
    z.append(data.iloc[i+4,-1])
    z.append(data.iloc[i+4,3])
    y.append(z)
    inp.append(X)


In [149]:
inp = np.array(inp)
y=np.array(y)

In [150]:
m = bias+np.dot(w[0],inp[:,0].T)+np.dot(w[1],inp[:,1].T)+np.dot(w[2],inp[:,2].T)

In [151]:
math.sqrt(metrics.mean_squared_error(y.T,m))

/home/kishora/PycharmProjects/abc/venv/lib/python3.6/site-packages/sklearn/metrics/regression.py:243: RuntimeWarning: overflow encountered in square
  output_errors = np.average((y_true - y_pred) ** 2, axis=0,


inf